### Import packages

In [4]:
import os
import glob
import numpy  as np
import pandas as pd
import librosa
import librosa.display
import matplotlib.pyplot as plt
import ipywidgets as widgets
import soundfile as sf

### Choose file to inspect¶

In [5]:
directory='/Users/saroltagabulya/git/Orca/'
wavs=glob.glob('*.csv')
file=[]

dropdown=widgets.Dropdown(
    options=wavs,
    description='CSV files:',
    disabled=False,
)

def dropdown_eventhandler(change):
    file.clear()
    file.append(change.new)
    print(change.new)
dropdown.observe(dropdown_eventhandler, names='value')
display(dropdown)

Dropdown(description='CSV files:', options=('time_stamps.csv', 'time_stamps_2017-02-04--10-14-06--00-16-35--BC…

time_stamps_2017-02-04--10-14-06--00-16-35--BC.csv


### Read csv file as pandas

In [17]:
filename = directory + file[0]
time_stamps=pd.read_csv(filename, index_col=0)

# Also extract recording name for later use
recording_name=file[0][12:-4]

### Read in wav

In [18]:
y, sr = librosa.load(recording_name + '.wav')

### Onsets & offsets in csv are in seconds => Convert to samples by multiplying them with the fs

second * fs = samples


In [20]:
time_stamps['onset_samples']=time_stamps.onset.apply(lambda x: int(round(x*sr)))
time_stamps['offset_samples']=time_stamps.offset.apply(lambda x: int(round(x*sr)))

### Function for cropping wav and saving

In [32]:
try:
    os.mkdir('./call_segments/' + recording_name)
    
except Exception as e:
    print(e)

[Errno 17] File exists: './call_segments/2017-02-04--10-14-06--00-16-35--BC'


In [33]:
def crop_call(row, y, sr, recording_name):
    call_filename= recording_name + '_' + str(row.onset_samples) + '_' + str(row.offset_samples) + '.wav'
    call=y[row.onset_samples: row.offset_samples]
    call_path='./call_segments/' + recording_name +'/' +  call_filename
    sf.write(call_path, call, sr) 
    return call_path

In [34]:
time_stamps['call_wav']=time_stamps.apply(lambda x: crop_call(x, y, sr, recording_name), axis=1)

### Function for converting wavs into spectrograms

In [35]:
def open_wav(row):
    y, sr = librosa.load(row.call_wav)
    return y, sr, row.call_wav

In [36]:
def wav_to_spect(y, sr, call_wav):
    
    if len(y)>0:
        # Convert to spectogram 
        Y = librosa.stft(y)
        Ydb = librosa.amplitude_to_db(abs(Y))

        plt.imshow(Ydb, aspect=0.3, origin='lower')
        plt.xlabel('time(samples)')
        plt.ylabel('Hz')

        # Save spectograms
        plt.savefig(call_wav[:-3] + 'png')
        plt.close()
    
        return Ydb
    
    else: 
        return 'Error with wav'

In [37]:
time_stamps['spect']=time_stamps.apply(lambda x: wav_to_spect(*open_wav(x)), axis=1) 

### Save csv under new name

In [38]:
time_stamps.to_csv('call_data_' + recording_name + '.csv')